# Capstone Project - The Battle of Neighborhoods

### Introduction

New York, which serves as the busiest city in the U.S., has been regarded as the paradise for tourism and business. According to the statistics from the government, the tourism in 2019 contributed to 10.94 trillion dollar, which is about 11.05% of the GDP. Since tourism has been populared, exploring locations and business to fit into this trend is essential to be considered.

### Bussiness

We would employ venue types generated by the FourSquare API to find which types of categories are popular and which neighbourhoods to perform the business. Stakeholders and investors would make their decisions based on the information given.

### Data

We employ data from a geojson file and then combined with information from the FourSquare API to consturct a new dataframe. We would create an account to get requests from FourSquare website throught the usage of API.


### Methodology 

We would describe any exploratory data analysis thatin use, any inferential statistical testing that is performed, if any, and what machine learnings were used and why. To group venues into clusters that present which is the most popular and what types exist in that area, we employ clustering method and hope it would give the result presenting the successfully open bussiness around that area 

### Result

### Discussion

### Conclusion